# Extracting electrical features and plotting traces using BluePyEfe

This notebook will show you how to use BluePyEfe to extract electrical features from experimental voltage traces. For this example we will use the rat somatosensory cortex data available at: http://microcircuits.epfl.ch/#/article/article_4_eph

Author of this script: Tanguy Damart @ Blue Brain Project

In [1]:
import bluepyefe as bpefe
import matplotlib.pyplot as plt

## Using BluePyEfe as a bulk efeatures extraction tool

BluePyEfe is used to extract electrical features for groups of cells. This is done using the "Extractor" class.

The main requirement to use the extractor class is a configuration dictionary which contains all the metadata for our trace files as well as options specifying how the extraction process will happen. For the current example on two cells, the structure of this dictionary will be as follow:

In [2]:
import glob

# Get the path of the files for the first cell and build the metadata dictionaries
files_B6 = []
for i in range(181, 186):
    files_B6.append({
                    "dt": 0.00025,
                    "i_file": "B6/B6_Ch0_IDRest_{}.ibw".format(i),
                    "v_file": "B6/B6_Ch3_IDRest_{}.ibw".format(i),
                    "i_unit": "pA",
                    "t_unit": "s",
                    "v_unit": "mV",
                    "ordinal": str(i),
                    })

files_B8 = []
for i in range(145, 150):
    files_B8.append({
                    "dt": 0.00025,
                    "i_file": "B8/B8_Ch0_IDRest_{}.ibw".format(i),
                    "v_file": "B8/B8_Ch3_IDRest_{}.ibw".format(i),
                    "i_unit": "pA",
                    "t_unit": "s",
                    "v_unit": "mV",
                    "ordinal": str(i),
                    })

# Build the config dictionary
config = {"cells": {
                "B6": {
                        "etype": "etype",
                        "exclude": [[], []],
                        "experiments": {
                            "IDRest": {
                                "files": files_B6,
                                "location": "soma"
                            }
                        },
                        "ljp": 14.,
                        "v_corr": 0
                },
                "B8": {
                        "etype": "etype",
                        "exclude": [[], []],
                        "experiments": {
                            "IDRest": {
                                "files": files_B8,
                                "location": "soma"
                            }
                        },
                        "ljp": 14.,
                        "v_corr": 0
                }
          }, 
          "comment": [],
          "features": {"IDRest": ['Spikecount', 'mean_frequency', 'ISI_CV', 'AP1_amp']},
          "format": 'igor',
          "options": {"delay": 500,
                      "logging": False,
                      "nanmean": True,
                      "relative": True,
                      "target": [150, 200, 250, 300],
                      "tolerance": 20.,
                      "onoff": {"IDRest": [700, 2700]},
                      "expthreshold": ["IDRest"]},
          "path": "./exp_data/"}

The main purpose of the extractor class is to output efeatures and protocols 
json files that can then be used as input for single cell model building 
using BluePyOpt. To do so, the efeatures need to be averaged over the cells 
present in the dataset for different step current amplitude (specified in the
 'target' field above). Since the option "relative" is set to True, these 
 targets need to be defined as 
 percentages of the spiking threshold of the cells, here 150%, 200%, 250% and 300%.

In [3]:
    extractor = bpefe.Extractor('output', config)
    extractor.create_dataset()
    extractor.plt_traces()
    extractor.extract_features(threshold=-30.)
    extractor.mean_features()
    extractor.analyse_threshold()
    extractor.plt_features()
    extractor.feature_config_cells()
    extractor.feature_config_all()
    extractor.plt_features_dist()

The resulting figures and electrical features and protocols can be seen in the ./output/ directory.